In [2]:
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as po
import plotly.io as pio

In [2]:
# Version 1
mult_reg_results_n_v_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Video/correlation_hamming_regression_results.pkl')
mult_reg_results_n_mf_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Still_MF/correlation_hamming_regression_results.pkl')
mult_reg_results_n_ff_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Still_FF/correlation_hamming_regression_results.pkl')

mult_reg_results_p_v_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Prior/Video/correlation_hamming_regression_results.pkl')
mult_reg_results_p_s_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Prior/Still/correlation_hamming_regression_results.pkl')

v1_mult_reg_results = [mult_reg_results_n_v_1, mult_reg_results_n_mf_1, mult_reg_results_n_ff_1,mult_reg_results_p_v_1, mult_reg_results_p_s_1]
v1_mult_reg_results = pd.concat(v1_mult_reg_results)

In [3]:
# Version 2
mult_reg_results_n_v_2, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V2/Naive/Video/correlation_hamming_regression_results.pkl')
mult_reg_results_n_mf_2, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V2/Naive/Still_MF/correlation_hamming_regression_results.pkl')
mult_reg_results_n_ff_2, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V2/Naive/Still_FF/correlation_hamming_regression_results.pkl')

mult_reg_results_p_v_2, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V2/Prior/Video/correlation_hamming_regression_results.pkl')
mult_reg_results_p_s_2, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V2/Prior/Still/correlation_hamming_regression_results.pkl')

v2_mult_reg_results = [mult_reg_results_n_v_2, mult_reg_results_n_mf_2, mult_reg_results_n_ff_2,mult_reg_results_p_v_2, mult_reg_results_p_s_2]
v2_mult_reg_results = pd.concat(v2_mult_reg_results)

In [9]:
def visualize_significant_regressors(mult_reg_results, electrode_region, exp_type, stm_type, version):
    er_idx = mult_reg_results.electrode_region==electrode_region
    ex_idx = mult_reg_results.experiment_type==exp_type
    st_idx = mult_reg_results.stimuli_type==stm_type
    reg_df = mult_reg_results[ex_idx & st_idx & er_idx]
    
    if reg_df.empty:
        print("Empty df: " +exp_type+"_"+stm_type+"_"+electrode_region)
    else:
        sig_idx = multipletests(reg_df["p_value"], alpha=0.05, method='bonferroni')[0]
        reg_df.loc[:, 'significant'] = sig_idx.tolist()

        sig_reg_df = reg_df.loc[reg_df['significant']]
        
        if not sig_reg_df.empty:
            fig = px.line(sig_reg_df, 
                         x="time", y="beta_value",  color="model_name",
                        title=exp_type+" "+stm_type+" "+electrode_region+" timepoints where the regressors reached significance")

            fig.update_yaxes(title_text="Regressors")
            fig.update_xaxes(range=[-200, 600], title_text="Time ms (relative to stimulus onset)")
            
            fig.write_image("outputs/multiple-regression/V"+str(version)+"/"+exp_type+"_"+stm_type+"_"+electrode_region+"_significant_beta_vals.png")
            #po.plot(fig, filename = "outputs/multiple-regression/V"+str(version)+"/"+exp_type+"_"+stm_type+"_"+electrode_region+"_significant_beta_vals.html", auto_open=False)

In [10]:
v1_mult_reg_results = v1_mult_reg_results[v1_mult_reg_results.model_name != 'constant']
for electrode_region in v1_mult_reg_results.electrode_region.unique():
    for exp_type in v1_mult_reg_results.experiment_type.unique():
        for stm_type in v1_mult_reg_results.stimuli_type.unique():
            if exp_type == "prior" and stm_type == "still-mf":
                continue
            visualize_significant_regressors(v1_mult_reg_results, electrode_region, exp_type, stm_type, 1)

In [11]:
v2_mult_reg_results = v2_mult_reg_results[v2_mult_reg_results.model_name != 'constant']
for electrode_region in v2_mult_reg_results.electrode_region.unique():
    for exp_type in v2_mult_reg_results.experiment_type.unique():
        for stm_type in v2_mult_reg_results.stimuli_type.unique():
            if exp_type == "prior" and stm_type == "still-mf":
                continue
            visualize_significant_regressors(v2_mult_reg_results, electrode_region, exp_type, stm_type, 2)

In [ ]:
#############################################################
#############################################################

In [ ]:
# Version 1
mult_rsa_results_n_v_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Video/correlation_hamming_avg_subject_results.pkl')
mult_rsa_results_n_mf_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Still_MF/correlation_hamming_avg_subject_results.pkl')
mult_rsa_results_n_ff_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Naive/Still_FF/correlation_hamming_avg_subject_results.pkl')

mult_rsa_results_p_v_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Prior/Video/correlation_hamming_avg_subject_results.pkl')
mult_rsa_results_p_s_1, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1/Prior/Still/correlation_hamming_avg_subject_results.pkl')

v1_mult_rsa_results = [mult_reg_results_n_v_1, mult_reg_results_n_mf_1, mult_reg_results_n_ff_1,mult_reg_results_p_v_1, mult_reg_results_p_s_1]
v1_mult_rsa_results = pd.concat(v1_mult_reg_results)

In [ ]:
# Plot on the correlational plot
avg_sbj_rsa_results, eeg_dist, model_dist  = pd.read_pickle('/auto/data2/oelmas/RSA_Results_V1//Naive/Video/correlation_hamming_avg_subjects_results.pkl')
rdm_df = avg_sbj_results

In [ ]:
def visualize_rsa_regression(rsa_df,mult_reg_results,model_name,electrode_region,exp_type,stm_type,version):
    
    # Get specified regression reesults
    er_idx = mult_reg_results.electrode_region==electrode_region
    ex_idx = mult_reg_results.experiment_type==exp_type
    st_idx = mult_reg_results.stimuli_type==stm_type
    md_idx = mult_reg_results.model_name==model_name
    reg_df = mult_reg_results[ex_idx & st_idx & er_idx & md_idx]
    
    df_to_plot = rdm_df[(rdm_df["model_name"] == model_name) 
                    & (rdm_df["electrode_region"] == electrode_region)
                    & (rdm_df["experiment_type"] == exp_type) 
                    & (rdm_df["stimulus_type"] == stm_type)]
    
    fig = go.Figure()

    # Plots the kendall tau curve
    fig.add_trace(go.Scatter(x=df_to_plot["time"],
                            y=df_to_plot["kendall_tau"],
                            mode='lines'))


    fig.add_shape(
            # Rectangle reference to the plot
                type="rect",
                xref="x",
                yref="paper",
                x0=min(reg_df.loc[reg_df['significant']].time),
                y0=0,
                x1=max(reg_df.loc[reg_df['significant']].time),
                y1=1,
                opacity=0.4,
                layer="below",
                line_width=0,
                fillcolor="LightSalmon",
                name='Model reached significance in the multiple regression analysis',
            )

    fig.update_yaxes(title_text="Kendall-tau")
    fig.update_xaxes(range=[-200, 600], title_text="Time ms (relative to stimulus onset)")

    fig.update_layout(
        height=800,
        title_text='Correlation of the ' + model_name + ' model',
        showlegend=True
    )

    po.plot(fig, filename = "outputs/multiple-regression/TEST.html", auto_open=False)

In [ ]:
#############################################################
#############################################################
# variance inflation factor
VIF =  pd.read_csv('/auto/data2/oelmas/RSA_Results_V1//Naive/Video/VIF_Results.csv', index_col=[0])
print(VIF)

In [3]:
mult_reg_results_n_v_1, _,_ = pd.read_pickle('/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Naive/Still/correlation_hamming_regression_results.pkl')

{'central': [0.009404490191604165,
  0.001200594699110935,
  0.0024376838989074168,
  0.006756075980963727,
  0.004071436497493419,
  -0.0008950915470873344,
  -0.0006277356657253819,
  0.003441218744224095,
  0.0002456034547333452,
  0.006323966561884609,
  0.011162745086167103,
  0.005283416451489442,
  0.015576266183639054,
  0.015611868243385674,
  0.0063450372367998,
  -0.002793731565069457,
  -0.010358332046631613,
  -0.012517244977887199,
  -0.008762876032520017,
  -0.01005759953761376,
  -0.013241167910594376,
  -0.007412595385192011,
  -0.0063440409827086874,
  -0.0026766352925502446,
  0.0008659442565345232,
  0.010039977576249526,
  0.03079830682783291,
  0.015471883440079326,
  -0.0010111399598915494,
  -0.010064722837188844,
  -0.009941486245231834,
  -0.004269526417550518,
  0.011152633351957153,
  -0.007603793055924957,
  0.0066582960678780445,
  0.004336629854202578,
  0.011725280111692715,
  0.008589028710349389,
  0.0007544875420378849,
  -0.0029219444668551198,
  -0.